In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import losses, optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, MaxPooling2D, Conv2D, InputLayer, ReLU, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
    
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Set a seed value
seed_value= 12321 

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

In [4]:
def read_images(folder):
    images = []
    for filename in tqdm(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) ##convert images from BGR to RGB
        if img is not None:
            img = cv2.resize(img, dsize = (70,140), interpolation=cv2.INTER_CUBIC)
            y,h,x,w = 0,100,0,70
            img = img[y:y+h, x:x+w]
            img = img/255
            images.append(img)

    return images

In [5]:
train_notflip = read_images("../images/training/0")
train_flip = read_images("../images/training/1")

X_noflip = np.array(train_notflip)
X_flip = np.array(train_flip)

y_noflip = np.zeros(X_noflip.shape[0])
y_flip = np.ones(X_flip.shape[0])

X = np.concatenate((X_noflip,X_flip ))
y = np.concatenate((y_noflip, y_flip))

X.shape, y.shape

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../images/training/0'

In [ ]:
test_notflip = read_images("../images/testing/0")
test_flip = read_images("../images/testing/1")

X_test_noflip = np.array(test_notflip)
X_test_flip = np.array(test_flip)
y_test_noflip = np.zeros(X_test_noflip.shape[0])
y_test_flip = np.ones(X_test_flip.shape[0])


X_test = np.concatenate((X_test_noflip,X_test_flip ))
y_test = np.concatenate((y_test_noflip, y_test_flip))
X_test.shape, y_test.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y , test_size = 0.2, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# visualizing the image
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.subplot(1,1,1)
plt.imshow(X_train[1], cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(X[i])
    plt.title(y[i])
    plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(-6, 0):
    ax = plt.subplot(3, 3, 0 - i)
    plt.imshow(X[i])
    plt.title(y[i])
    plt.axis("off")

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), activation = ReLU(), padding = 'same', input_shape=(100, 70,3)))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(64, activation=ReLU()))
model.add(Dense(1, activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.001)

model.compile(loss=losses.binary_crossentropy, optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_model = model.fit(X_train, y_train, validation_data=(X_val, y_val), shuffle=True, epochs=12, batch_size=64, verbose=1)

In [ ]:
# plotting the accuracies

dict_hist = history_model.history
list_ep = [i for i in range(1,13)]

plt.figure(figsize = (8,8))
plt.plot(list_ep,dict_hist['accuracy'],ls = '--', label = 'accuracy')
plt.plot(list_ep,dict_hist['val_accuracy'
                          ],ls = '--', label = 'val_accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
test_pred = model.predict(X_test, verbose=1)


# Choose a threshold of 0.5
threshold = 0.5

# Convert probabilities to predicted labels
test_pred_labels = np.where(test_pred >= threshold, 1, 0)
#test_pred_labels1"

In [ ]:
#Printing the classification report
print(classification_report(y_test, test_pred_labels))

#Plotting the heatmap using confusion matrix
cm = confusion_matrix(y_test, test_pred_labels)
plt.figure(figsize=(8,5))
sns.heatmap(cm, annot=True,  fmt='.0f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
model.save("../models/ggg.h5")

In [ ]:
## 3 files read process, train inference, then we need file dor pre trained model\n",

import cv2
import os
from tqdm import tqdm

class ImageProcessor:
    def __init__(self, size=(70, 140)):
        self.size = size
        self.crop_coords = (0, 100, 0, 70)

    def process(self, folder):
        images = []
        for filename in tqdm(os.listdir(folder)):
            img = cv2.imread(os.path.join(folder,filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) ##convert images from BGR to RGB
            if img is not None:
                img = cv2.resize(img, dsize=self.size, interpolation=cv2.INTER_CUBIC)
                y, h, x, w = self.crop_coords
                img = img[y:y+h, x:x+w]
                img = img/255
                images.append(img)



        return images

In [ ]:
## Train the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
from tensorflow.keras import losses, optimizers
import numpy as np

class CNNModel:
    def __init__(self, input_shape=(100, 70, 3)):
        self.model = Sequential()
        self.model.add(Conv2D(32, (2, 2), activation=ReLU(), padding='same', input_shape=input_shape))
        self.model.add(MaxPooling2D((2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation=ReLU()))
        self.model.add(Dense(1, activation='sigmoid'))
        adam = optimizers.Adam(learning_rate=0.001)
        self.model.compile(loss=losses.binary_crossentropy, optimizer=adam, metrics=['accuracy'])

    def train(self, X_train, y_train, X_val, y_val, epochs=12, batch_size=64, verbose=1):
        self.history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), shuffle=True, epochs=epochs, batch_size=batch_size, verbose=verbose)

    def predict(self, X):
        return self.model.predict(X) 

In [ ]:
processor = ImageProcessor()

X_noflip =np.array(processor.process('../images/training/0'))
X_flip = np.array(processor.process('../images/training/1'))

y_noflip = np.zeros(X_noflip.shape[0])
y_flip = np.ones(X_flip.shape[0])

X = np.concatenate((X_noflip,X_flip ))
y = np.concatenate((y_noflip, y_flip))

X.shape, y.shape

In [ ]:
X_test_noflip = np.array(processor.process('../images/testing/0'))
X_test_flip = np.array(processor.process('../images/testing/1'))

y_test_noflip = np.zeros(X_test_noflip.shape[0])
y_test_flip = np.ones(X_test_flip.shape[0])


X_test = np.concatenate((X_test_noflip,X_test_flip ))
y_test = np.concatenate((y_test_noflip, y_test_flip))
X_test.shape, y_test.shape

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y , test_size = 0.2, shuffle=True)

In [ ]:
cnn = CNNModel()
cnn.train(X_train, y_train, X_val, y_val)

In [ ]:
test_pred = cnn.predict(X_test)


# Choose a threshold of 0.5
threshold = 0.5

# Convert probabilities to predicted labels
test_pred_labels = np.where(test_pred >= threshold, 1, 0)
#test_pred_labels1

In [ ]:
#Printing the classification report
print(classification_report(y_test, test_pred_labels))

#Plotting the heatmap using confusion matrix
cm = confusion_matrix(y_test, test_pred_labels)
plt.figure(figsize=(8,5))
sns.heatmap(cm, annot=True,  fmt='.0f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
print(classification_report(y_test, test_pred_labels))